In [1]:
from src.crawler import WebCrawler
from src.parser import VerbParser

crawler = WebCrawler()
parser = VerbParser()

url = "https://www.spanishdict.com/conjugate/hablar"


In [ ]:
html = crawler.crawl_page(url)
